In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

from torchsummary import summary

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import pandas as pd
import random
from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split
import time



In [34]:
datadir = 'data'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', None)
data.head()

center  \
0  /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/center_2022_10_30_20_43_12_593.jpg   
1  /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/center_2022_10_30_20_43_12_698.jpg   
2  /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/center_2022_10_30_20_43_12_799.jpg   
3  /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/center_2022_10_30_20_43_12_904.jpg   
4  /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/center_2022_10_30_20_43_13_006.jpg   

                                                                                             left  \
0   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/left_2022_10_30_20_43_12_593.jpg   
1   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/left_2022_10_30_20_43_12_698.jpg   
2   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/left_2022_10_30_20_43_12_799.jpg   
3   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/left_2022_10_30_20_43_12_904.jpg   
4   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/left_2022_10_30_20_43_13_006.jpg   

                                                                                             right  \
0   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/right_2022_10_30_20_43_12_593.jpg   
1   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/right_2022_10_30_20_43_12_698.jpg   
2   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/right_2022_10_30_20_43_12_799.jpg   
3   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/right_2022_10_30_20_43_12_904.jpg   
4   /Users/liweijie/Desktop/Perception_Proj/network_dev/data/IMG/right_2022_10_30_20_43_13_006.jpg   

   steering  throttle  reverse     speed  
0       0.0       0.0        0  0.000079  
1       0.0       0.0        0  0.000081  
2       0.0       0.0        0  0.000079  
3       0.0       0.0        0  0.000080  
4       0.0       0.0        0  0.000078

In [35]:
def load_img_steering(datadir, df):
    image_path = []
    steering = []
    for i in range(len(data)):
        indexed_data = data.iloc[i]
        center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
        image_path.append(os.path.join(datadir, center.strip()))
        steering.append(float(indexed_data[3]))
        # left image append
        image_path.append(os.path.join(datadir,left.strip()))
        steering.append(float(indexed_data[3])+0.15)
        # right image append
        image_path.append(os.path.join(datadir,right.strip()))
        steering.append(float(indexed_data[3])-0.15)
    image_paths = np.asarray(image_path)
    steerings = np.asarray(steering)
    return image_paths, steerings
 

In [36]:
def pan(image):
    pan = iaa.Affine(translate_percent= {"x" : (-0.1, 0.1), "y": (-0.1, 0.1)})
    image = pan.augment_image(image)
    return image

def img_random_brightness(image):
    brightness = iaa.Multiply((0.2, 1.2))
    image = brightness.augment_image(image)
    return image

def zoom(image):
    zoom = iaa.Affine(scale=(1, 1.3))
    image = zoom.augment_image(image)
    return image

def img_random_flip(image, steering_angle):
    image = cv2.flip(image,1) # one for horizontal flip
    steering_angle =- steering_angle
    return image,steering_angle
    
def random_augment(image, steering_angle):
    image = mpimg.imread(image)
    if np.random.rand() < 0.5:
        image = pan(image)
    if np.random.rand() < 0.5:
        image = zoom(image)
    if np.random.rand() < 0.5:
        image = img_random_brightness(image)
    if np.random.rand() < 0.5:
        image, steering_angle = img_random_flip(image, steering_angle)
    
    return image, steering_angle

def img_preprocess(img):
    img = img[60:135,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

# def batch_generator(image_paths, steering_ang, batch_size, istraining):
  
#     while True:
#         batch_img = []
#         batch_steering = []
    
#         for i in range(batch_size):
#             random_index = random.randint(0, len(image_paths) - 1)
      
#             if istraining:
#                 im, steering = random_augment(image_paths[random_index], steering_ang[random_index])
     
#             else:
#                 im = mpimg.imread(image_paths[random_index])
#                 steering = steering_ang[random_index]
      
#             im = img_preprocess(im)
#             batch_img.append(im)
#             batch_steering.append(steering)
#         yield (np.asarray(batch_img), np.asarray(batch_steering)) 

In [212]:

class DrivingImageDataset(Dataset):
    def __init__(self, image_paths, steering_ang, transform=None):
        self.img_paths = image_paths
        self.steering_angs = steering_ang
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = mpimg.imread(img_path)
        img = img_preprocess(img)
        img = img.astype(np.uint8)
        steering = self.steering_angs[index]

        if self.transform is not None:
            img = self.transform(img)
            # img = img.to(dtype=torch.long)
            steering = torch.tensor(steering)
            steering = steering.to(dtype=torch.float32)

        # print('steering: ',steering.shape)
        return img, steering

In [102]:

class nvidia_model(nn.Module):
    in_planes = [3, 24, 36, 48, 64, 64, 1152, 100, 50, 10, 1]
    kernel_size = [5, 5, 5, 3, 3]
    dropout_p = [0.45, 0.4, 0.4]
    def __init__(self):
        super(nvidia_model, self).__init__()
        self.bn1 = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(self.in_planes[0], self.in_planes[1], self.kernel_size[0], stride=2)
        self.conv2 = nn.Conv2d(self.in_planes[1], self.in_planes[2], self.kernel_size[1], stride=2)
        self.conv3 = nn.Conv2d(self.in_planes[2], self.in_planes[3], self.kernel_size[2], stride=2)
        self.conv4 = nn.Conv2d(self.in_planes[3], self.in_planes[4], self.kernel_size[3])
        self.conv5 = nn.Conv2d(self.in_planes[4], self.in_planes[5], self.kernel_size[4])
        self.dropout1 = nn.Dropout2d(p=self.dropout_p[0])
        self.fc1 = nn.Linear(self.in_planes[6], self.in_planes[7])
        self.dropout2 = nn.Dropout2d(p=self.dropout_p[1])
        self.fc2 = nn.Linear(self.in_planes[7], self.in_planes[8])
        self.dropout3 = nn.Dropout2d(p=self.dropout_p[2])
        self.fc3 = nn.Linear(self.in_planes[8], self.in_planes[9])
        self.output = nn.Linear(self.in_planes[9], self.in_planes[10])

    # def print_layer(self, layer):
    #     print(layer.shape)
    #     print(type(layer))

    def forward(self, x):
        # self.print_layer(x)
        out = self.bn1(x)
        # self.print_layer(out)
        out = F.elu(self.conv1(out))
        out = F.elu(self.conv2(out))
        out = F.elu(self.conv3(out))
        out = F.elu(self.conv4(out))
        out = F.elu(self.conv5(out))
        out = self.dropout1(out)
        out = out.view(out.size(0), -1)
        out = F.elu(self.fc1(out))
        out = self.dropout2(out)
        out = F.elu(self.fc2(out))
        out = self.dropout3(out)
        out = F.elu(self.fc3(out))
        out = self.output(out)
        return out



In [74]:
TOTAL_BAR_LENGTH = 65.
last_time = time.time()
begin_time = last_time
term_width = 80 

def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds*1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f

def progress_bar(current, total, msg=None):
    global last_time, begin_time
    if current == 0:
        begin_time = time.time()  # Reset for new bar.

    cur_len = int(TOTAL_BAR_LENGTH*current/total)
    rest_len = int(TOTAL_BAR_LENGTH - cur_len) - 1

    sys.stdout.write(' [')
    for i in range(cur_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    cur_time = time.time()
    step_time = cur_time - last_time
    last_time = cur_time
    tot_time = cur_time - begin_time

    L = []
    L.append('  Step: %s' % format_time(step_time))
    L.append(' | Tot: %s' % format_time(tot_time))
    if msg:
        L.append(' | ' + msg)

    msg = ''.join(L)
    sys.stdout.write(msg)
    for i in range(term_width-int(TOTAL_BAR_LENGTH)-len(msg)-3):
        sys.stdout.write(' ')

    # Go back to the center of the bar.
    for i in range(term_width-int(TOTAL_BAR_LENGTH/2)+2):
        sys.stdout.write('\b')
    sys.stdout.write(' %d/%d ' % (current+1, total))

    if current < total-1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()

In [237]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    # Setup time measurement for training epoch
    _tTrainAcc = 0
    _cnt = 0
    _topAccu = 0
    _tCommAccu = 0
    _tComputeAccu = 0
    _tBegin = time.perf_counter()

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        _tTrainStart = time.perf_counter()
        inputs, targets = inputs.to(device), targets.to(device)
        # print('Test: ',targets.shape)
        # inputs = inputs.float()
        # targets = targets.float()
        # print(inputs.dtype)
        optimizer.zero_grad()
        outputs = net(inputs)
        output_cri = torch.squeeze(outputs)

        # print(outputs.shape)
        # print(inputs.shape)
        # print(targets.shape)
        # break
        # print(outputs.dtype)
        # print(targets.dtype)

        # inputs = inputs.Long()


        loss = criterion(output_cri, targets)
        # print(loss.dtype)

        _tCommStart = time.perf_counter()
        loss.backward()
        optimizer.step()
        _tTrainEnd = time.perf_counter()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if not verboseFlag:
            progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                    % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        _tTrainAcc += _tTrainEnd - _tTrainStart
        _tCommAccu += _tTrainEnd- _tCommStart
        _tComputeAccu += _tCommStart - _tTrainStart

    _tEnd = time.perf_counter()
    totalTime = _tEnd - _tBegin
    trainTime = _tTrainAcc
    commTime = _tCommAccu
    computeTime = _tComputeAccu
    loadTime = totalTime - trainTime
    return train_loss, trainTime, totalTime, loadTime, commTime, computeTime

        



In [184]:
model = nvidia_model()
summary(model, (3,66,200), batch_size=200)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1          [200, 3, 66, 200]               6
            Conv2d-2          [200, 24, 31, 98]           1,824
            Conv2d-3          [200, 36, 14, 47]          21,636
            Conv2d-4           [200, 48, 5, 22]          43,248
            Conv2d-5           [200, 64, 3, 20]          27,712
            Conv2d-6           [200, 64, 1, 18]          36,928
         Dropout2d-7           [200, 64, 1, 18]               0
            Linear-8                 [200, 100]         115,300
         Dropout2d-9                 [200, 100]               0
           Linear-10                  [200, 50]           5,050
        Dropout2d-11                  [200, 50]               0
           Linear-12                  [200, 10]             510
           Linear-13                   [200, 1]              11
Total params: 252,225
Trainable params:

In [239]:

if __name__ == "__main__":
    verboseFlag = False
    epochs = 20
    print('==> Preparing Data')
    transform_train = transforms.Compose([
        # transforms.RandomCrop(32, padding=4),
        # transforms.RandomHorizontalFlip(),
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])

    transform_valid = transforms.Compose([
        transforms.ToTensor()
    ])

    image_paths, steerings = load_img_steering(datadir + '/IMG', data)
    X_train, X_valid, y_train, y_valid = train_test_split(image_paths, steerings, test_size=0.2)
    print('Training Samples: {}\nValid Samples: {}'.format(len(X_train), len(X_valid)))

    trainset = DrivingImageDataset(X_train, y_train, transform_train)
    testset = DrivingImageDataset(X_valid, y_valid, transform_valid)

    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size = 200, shuffle=True
    )
    testloader = torch.utils.data.DataLoader(
        testset, batch_size = 100, shuffle=True
    )

    print('==> Building Model')
    device = 'cpu'
    # device = 'cuda'
    net = nvidia_model()
    # net = net.double()
    # summary(net)
    net = net.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
    for epoch in range(0, epochs):
        trainPerf = train(epoch)
        scheduler.step()

    
    


==> Preparing Data
Training Samples: 3220
Valid Samples: 806
==> Building Model

Epoch: 0
 [=============================================================>...]  Step: 120ms | Tot: 17s56ms | Loss: 0.040 | Acc: 24.503% (789/322 17/17 7  

Epoch: 1
 [=============================================================>...]  Step: 126ms | Tot: 16s478ms | Loss: 0.038 | Acc: 24.503% (789/322 17/17   

Epoch: 2
 [=============================================================>...]  Step: 120ms | Tot: 16s513ms | Loss: 0.036 | Acc: 24.503% (789/322 17/17   

Epoch: 3
 [=============================================================>...]  Step: 129ms | Tot: 17s895ms | Loss: 0.039 | Acc: 24.503% (789/322 17/17 7 

Epoch: 4
 [=============================================================>...]  Step: 156ms | Tot: 19s645ms | Loss: 0.038 | Acc: 24.503% (789/322 17/17 7 

Epoch: 5
 [=============================================================>...]  Step: 126ms | Tot: 19s308ms | Loss: 0.037 | Acc: 24.503% (789/322

In [240]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([0.6889, 0.6889, 0.6889], requires_grad=True)
Parameter containing:
tensor([-0.0048, -0.0044,  0.0029], requires_grad=True)
Parameter containing:
tensor([[[[-1.4846e-05,  2.1516e-10, -1.7177e-06, -1.2087e-08, -7.4928e-11],
          [ 2.4262e-07, -5.5461e-10,  3.9660e-08,  6.9315e-10,  9.5741e-10],
          [-6.3284e-09, -2.6768e-08, -1.5562e-09, -1.0460e-09, -1.0292e-07],
          [-1.4734e-08,  1.1652e-11, -3.1274e-06,  3.5593e-12, -1.8698e-07],
          [ 1.6447e-05,  4.7290e-05,  1.9037e-07, -8.1148e-10, -1.1792e-09]],

         [[-4.7908e-11, -9.7930e-10,  3.2180e-05, -1.0705e-06,  9.7097e-10],
          [-1.7646e-07,  1.9675e-09,  3.4633e-10,  3.6220e-11,  6.5148e-09],
          [-7.3400e-10,  1.9661e-10,  1.9048e-09, -8.5675e-10, -5.6450e-08],
          [ 5.7502e-05,  7.4594e-11, -3.6715e-09,  1.8931e-09,  3.2596e-06],
          [ 8.1169e-10, -6.6809e-08,  1.6560e-10,  9.2852e-09,  1.2735e-09]],

         [[ 4.5334e-10, -2.2892e-06, -6.0162e-09, -